In [1]:
from google.colab import drive
drive.mount('/content/drive')
cd='/content/drive/My Drive/Colab Notebooks/'
%cd /content/drive/My Drive/Colab Notebooks/
import os, cv2

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks


In [2]:
# Keras Models
import keras 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

# Aditional Libs
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD

In [3]:
data_path = '/content/drive/MyDrive/Colab Notebooks/CK+48'
data_dir_list = os.listdir(data_path)
img_data_list=[]
labels_list=[]
class_value=0 #i is the label 
for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
    for img in img_list:
        # Using 0 to read image in grayscale mode
        input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img, 0 )
        #input_img=cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
        #input_img_resize=cv2.resize(input_img,(48,48))
        img_data_list.append(input_img)
        labels_list.append(class_value)
    class_value+=1
        
img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data = img_data/np.amax(img_data)#normalize data by dividing each pixel value by the maximum
img_data.shape #The dataset contains 750 images, each image is a 2D shape 48x48

Loaded the images of dataset-anger

Loaded the images of dataset-surprise

Loaded the images of dataset-happy

Loaded the images of dataset-sadness

Loaded the images of dataset-fear



(750, 48, 48)

In [4]:
labels = np.array(labels_list)
labels = labels.astype('int32')
print(labels.shape)

(750,)


In [51]:
x_train = img_data[:720]
x_test = img_data[721:]
y_train = labels[:720]
y_test = labels[721:]
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.33)
x_train = x_train.reshape(len(x_train),48,48)
x_valid = x_valid.reshape(len(x_valid),48,48)
y_train = to_categorical(y_train, 5)
y_valid = to_categorical(y_valid, 5)
y_test = to_categorical(y_test, 5)

In [52]:
# Model is the MLP network that we want to create
model = Sequential()
model.add(Flatten(input_shape=(48,48)))
model.add(Dense(256, activation='tanh', kernel_initializer = 'he_normal'))
model.add(Dropout(0.4))
model.add(Dense(128, activation='tanh', kernel_initializer = 'he_normal'))
model.add(Dropout(0.4))
model.add(Dense(100, activation='tanh', kernel_initializer = 'he_normal'))
model.add(Dropout(0.4))
model.add(Dense(5, activation='softmax', kernel_initializer = 'he_normal'))
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_13 (Flatten)         (None, 2304)              0         
_________________________________________________________________
dense_58 (Dense)             (None, 256)               590080    
_________________________________________________________________
dropout_45 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_59 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_46 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_60 (Dense)             (None, 100)               12900     
_________________________________________________________________
dropout_47 (Dropout)         (None, 100)             

In [53]:
optimizer = SGD(learning_rate=0.01, momentum=0.975, decay=2e-06, nesterov=True)
model.compile(loss='categorical_crossentropy',
			  optimizer=optimizer,
			  metrics=['accuracy'])

history = model.fit(x_train, y_train,
					batch_size=64,
					epochs=100,
					verbose=2,
					validation_data=(x_valid, y_valid))

Epoch 1/100
8/8 - 1s - loss: 2.0260 - accuracy: 0.2988 - val_loss: 1.3011 - val_accuracy: 0.3866
Epoch 2/100
8/8 - 0s - loss: 1.7483 - accuracy: 0.3008 - val_loss: 1.3165 - val_accuracy: 0.3824
Epoch 3/100
8/8 - 0s - loss: 1.5895 - accuracy: 0.3838 - val_loss: 1.2088 - val_accuracy: 0.4664
Epoch 4/100
8/8 - 0s - loss: 1.3811 - accuracy: 0.4564 - val_loss: 1.0298 - val_accuracy: 0.6176
Epoch 5/100
8/8 - 0s - loss: 1.3089 - accuracy: 0.5021 - val_loss: 0.9541 - val_accuracy: 0.6261
Epoch 6/100
8/8 - 0s - loss: 1.1179 - accuracy: 0.5996 - val_loss: 0.8605 - val_accuracy: 0.6471
Epoch 7/100
8/8 - 0s - loss: 0.9597 - accuracy: 0.6473 - val_loss: 0.7679 - val_accuracy: 0.7017
Epoch 8/100
8/8 - 0s - loss: 0.8980 - accuracy: 0.6494 - val_loss: 0.6926 - val_accuracy: 0.7437
Epoch 9/100
8/8 - 0s - loss: 0.8723 - accuracy: 0.6556 - val_loss: 0.6993 - val_accuracy: 0.7269
Epoch 10/100
8/8 - 0s - loss: 0.8163 - accuracy: 0.7033 - val_loss: 0.5609 - val_accuracy: 0.7983
Epoch 11/100
8/8 - 0s - loss:

In [54]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test top 1 accuracy:', score[1])

Test loss: 0.6188495755195618
Test top 1 accuracy: 0.7931034564971924
